In [1]:
#| default_exp data_processing
#|  export
from fastcore.script import call_parse
from ast import literal_eval as make_tuple
from torchtext.data.utils import get_tokenizer
import torch
from torchtext.vocab import build_vocab_from_iterator
from typing import Any
# @call_parse
# def split_string(string):
#     # Removing the parentheses and splitting the string by comma
#     parts = string[1:-1].split(",")
#     # Removing the whitespace and quotes from the parts
#     parts = [part.strip().strip("'") for part in parts]
#     return parts[0], parts[1]

def yield_tokens(
        data_iter:Any # Iterable database instance
):
    tokenizer = get_tokenizer('basic_english')
    for _, text in data_iter:
        yield tokenizer(text)

#
# @call_parse
# def return_iters(db:str # Path to db
#                  ):
#     train_iter = []
#     test_iter = []
#     file = open(db, 'r', encoding='latin1')
#     mapping = {
#         "Libertarian Left": 1,
#         "Libertarian Right": 2,
#         "Authoritarian Left": 3,
#         "Authoritarian Right": 4,
#     }
#     lines = file.readlines()
#     for line in lines:
#         opinion,text = split_string(line)
#         train_iter+=[(mapping[opinion],text)]
#         test_iter+=[(mapping[opinion],text)]
#     train_iter = iter(train_iter)
#     test_iter = iter(test_iter)
#     file.close()
#     return train_iter, test_iter

def split_string(string):
    # Removing the parentheses and splitting the string by comma
    parts = string[1:-1].split(",")
    # Removing the whitespace and quotes from the parts
    parts = [part.strip().strip("'") for part in parts]
    return parts[0], parts[1]

def return_iters(db:str # Path to db
                 ):
    train_iter = []
    test_iter = []
    file = open(db, 'r', encoding='latin1')
    mapping = {
        "Libertarian Left": 1,
        "Libertarian Right": 2,
        "Authoritarian Left": 3,
        "Authoritarian Right": 4,
    }
    lines = file.readlines()
    for line in lines:
        opinion,text = split_string(line)
        train_iter+=[(mapping[opinion],text)]
        test_iter+=[(mapping[opinion],text)]
    train_iter = iter(train_iter)
    test_iter = iter(test_iter)
    file.close()
    return train_iter, test_iter
def collate_batch(
        batch
):

    db="../uniqueDB.txt" # Path to database
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_iter, test_iter = return_iters(db)
    label_pipeline = lambda x: int(x) - 1
    tokenizer = get_tokenizer('basic_english')
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)